# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1uAUJGEUzfNj6OsWNAimnYCw7eKaHhMUfU1MTj9YwYw4/edit?usp=sharing), [grading rubric](https://docs.google.com/document/d/1hKuRWqFcIdhOkow3Nljcm7PXzIkoa9c_aHkMKZDxWa0/edit?usp=sharing)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an outline to help you with your own approach.**_

## Project Setup

In [ ]:
import math
import os
import numpy as np
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
from sqlalchemy import create_engine
import re
import datetime
import sqlite3

In [ ]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "data/taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
UBER_CSV = ""
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [ ]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Calculate distance

In [ ]:
def calculate_distance_with_coords(pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude):
    lat1=pickup_longitude
    lon1=pickup_latitude
    lat2=dropoff_longitude
    lon2=dropoff_latitude
    
    R = 6371  # Earth's radius in kilometers

    # convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # calculate differences between latitudes and longitudes
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # calculate Haversine formula
    a = math.sin(d_lat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c

    return distance

Add Distance Column

In [ ]:
def add_distance_column(dataframe):
    dataframe['distance'] = dataframe.apply(lambda row: calculate_distance_with_coords(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
    return dataframe

### Process Taxi Data

In [ ]:
# Get the website page of the taxi_url
def get_taxi_html():
    response = requests.get(TAXI_URL)
    html = response.content
    return html

In [ ]:
# In the website page, find links for 'Yellow Taxi Trip Records'
# From 2009 to 2015 monthly data
def find_taxi_parquet_links():
    get_taxi_html()
    soup = bs4.BeautifulSoup(get_taxi_html(), 'html.parser')
    l1 = soup.find_all("a")
    l2 = []
    l3 = []
    pattern = []
    for i in range(len(l1)):
        if l1[i].text == 'Yellow Taxi Trip Records':
            l2.append(l1[i]['href'])
    for j in range(len(l2)):
        pattern.append(r"2009-\d{2}")
        pattern.append(r"2010-\d{2}")
        pattern.append(r"2011-\d{2}")
        pattern.append(r"2012-\d{2}")
        pattern.append(r"2013-\d{2}")
        pattern.append(r"2014-\d{2}")
        pattern.append(r"2015-\d{2}")
        for i in range(7):
            if re.search(pattern[i], l2[j]):
                l3.append(l2[j])
                break
        
    return l3

In [ ]:
# Get the taxi zones data, process the data to show the longitude and latitude for a given location ID
taxi_zones_df = gpd.read_file("taxi_zones/taxi_zones.shp")
taxi_zones_df = taxi_zones_df.to_crs(CRS)
taxi_zones_df['longitude'] = taxi_zones_df.centroid.x  
taxi_zones_df['latitude'] = taxi_zones_df.centroid.y

In [ ]:
# Given the url of the 'Yellow Taxi Trip Records', download the data, 
# Generate a sampling of Yellow Taxi data that’s roughly equal to the sample size of the Uber dataset
# Select useful columns and rename them
# Calculate duration of trip (trip end time - trip start time)
# If the dataset only have location ID, select rows with valid location ID
# change the location ID into longitude and latitude
# Remove the data that is not within the specified limits
# Change the measurement of duration to seconds, delete rows that have super long duration (outlier)
# Delete rows with negative duration
# Delete rows that have super long distance (outlier)
# Delete rows with negative distance
# Return the cleaned dataframe

def process_dataframe(url, taxi_zones_df):
    df = pd.read_parquet(url, engine='pyarrow')
    df = df.sample(frac =0.0002)
    if "tpep_pickup_datetime" not in df.columns:
        if "Trip_Pickup_DateTime" in df.columns:
            df = df[["Trip_Pickup_DateTime", "Trip_Dropoff_DateTime", "Start_Lon", "Start_Lat", "End_Lon", "End_Lat", "Tip_Amt"]]
            df.rename(columns = {'Start_Lon':'pickup_longitude', 'Start_Lat':'pickup_latitude',
                                  'End_Lon':'dropoff_longitude', 'End_Lat':'dropoff_latitude'}, inplace = True)

        elif "pickup_datetime" in df.columns:
            df = df[["pickup_datetime", "dropoff_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "tip_amount"]]
            df.rename(columns = {'pickup_datetime':'Trip_Pickup_DateTime', 'dropoff_datetime':'Trip_Dropoff_DateTime',
                                  'tip_amount':'Tip_Amt'}, inplace = True)  
        
        df['Trip_Pickup_DateTime'] = df['Trip_Pickup_DateTime'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        df['Trip_Dropoff_DateTime'] = df['Trip_Dropoff_DateTime'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        df['duration'] =  df['Trip_Dropoff_DateTime'] - df['Trip_Pickup_DateTime']
        
        
        
    else:
        df = df[["tpep_pickup_datetime", "tpep_dropoff_datetime", "PULocationID", "DOLocationID", "tip_amount"]]
        df = df[(df["PULocationID"] < 263) & (df["PULocationID"] > 0) 
               & (df["DOLocationID"] < 263) & (df["DOLocationID"] > 0)]
        df = df.merge(taxi_zones_df[['LocationID','longitude','latitude']].set_index('LocationID'),
                                    left_on='PULocationID', right_on='LocationID')
        df = df.rename(columns={'longitude': 'pickup_longitude', 'latitude': 'pickup_latitude'})
        df = df.merge(taxi_zones_df[['LocationID','longitude','latitude']].set_index('LocationID'),
                                    left_on='DOLocationID', right_on='LocationID')
        df = df.rename(columns={'longitude': 'dropoff_longitude', 'latitude': 'dropoff_latitude'})
        df = df.drop(columns=["PULocationID", "DOLocationID"])
        df = df.rename(columns={'tpep_pickup_datetime': 'Trip_Pickup_DateTime', 'tpep_dropoff_datetime': 'Trip_Dropoff_DateTime', 'tip_amount': 'Tip_Amt'})
        df['duration'] =  df['Trip_Dropoff_DateTime'] - df['Trip_Pickup_DateTime']
        

    westlimit=-74.242330; southlimit=40.560445; eastlimit=-73.717047; northlimit=40.908524
    df = df[(df['pickup_longitude']<eastlimit) & (df['pickup_longitude']>westlimit)
        & (df['pickup_latitude']<northlimit) & (df['pickup_latitude']>southlimit)
        & (df['dropoff_longitude']<eastlimit) & (df['dropoff_longitude']>westlimit)
        & (df['dropoff_latitude']<northlimit) & (df['dropoff_latitude']>southlimit)]
    df['duration'] = df['duration'].apply(lambda x:x/np.timedelta64(1, 's'))
    df = df[(df['duration'] <= 10000) & (df['duration'] > 0)]
    df['distance'] = df.apply(lambda row: calculate_distance_with_coords(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
    df = df[(df['distance'] <= 100) & (df['distance'] > 0)]
    
    return df

In [ ]:
# Getting the full dataset for taxi data
# Get dataframes from every month by function process_dataframe(url, taxi_zones_df) (defined above)
# concat them together by time order to make the full dataset

def get_full_data():
    urls = find_taxi_parquet_links()
    df = process_dataframe(urls[72], taxi_zones_df)
    for i in range(73, 84):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])
    for i in range(60, 72):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])       
    for i in range(48, 60):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])
    for i in range(36, 48):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])    
    for i in range(24, 36):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])    
    for i in range(12, 24):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])    
    for i in range(0, 6):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])     
    return df

In [ ]:
taxi_data = get_full_data()

In [ ]:
taxi_data.head()

### Processing Uber Data

In [ ]:
def load_and_clean_uber_data(csv_file):
    data_1 = pd.read_csv(csv_file)
    data_2 = data_1.drop(columns=['Unnamed: 0','key','fare_amount','passenger_count'])
    data_3 = data_2.dropna()
    
    # Define the range of acceptable latitude and longitude values
    min_lat, max_lat = 40.560445, 40.908524
    min_long, max_long = -74.242330, -73.717047
    
    # Remove trips that start and/or end outside of the latitude/longitude coordinate box
    data_4 = data_3.drop(
    index=data_3[
    (data_3['pickup_latitude'] < min_lat) & 
    (data_3['pickup_latitude'] > max_lat) &
    (data_3['pickup_longitude'] < min_long) &
    (data_3['pickup_longitude'] > max_long) &
    (data_3['dropoff_latitude'] < min_lat) &
    (data_3['dropoff_latitude'] > max_lat) &
    (data_3['dropoff_longitude'] < min_long) &
    (data_3['dropoff_longitude'] > max_long)
    ].index
)
    
    #normalizing and using appropriate column types for the respective data
    data_4['pickup_datetime'] = pd.to_datetime(data_3['pickup_datetime'])
    data_4 = data_4.rename(columns = {'pickup_datetime': 'Trip_Pickup_DateTime'})
    
    return data_4

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    weather_data=pd.read_csv(csv_file)
    weather_data['REPORT_TYPE'] = weather_data['REPORT_TYPE'].astype(str)
    weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])
    
    # Filter out the rows where 'REPORT_TYPE' column contains the string 'SOD'
    hourly_data = weather_data.loc[~weather_data['REPORT_TYPE'].str.contains('SOD')]

    # Select only the required columns and rename them
    hourly_data = hourly_data[['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']]
    hourly_data = hourly_data.rename(columns={'DATE': 'datetime', 'HourlyPrecipitation': 'hourly_precipitation', 'HourlyWindSpeed': 'hourly_wind_speed'})

    # Convert 'HourlyPrecipitation' and 'HourlyWindSpeed' columns to numeric datatype
    hourly_data[['hourly_precipitation', 'hourly_wind_speed']] = hourly_data[['hourly_precipitation', 'hourly_wind_speed']].apply(pd.to_numeric, errors='coerce')

    # Fill NaN values in 'HourlyPrecipitation' and 'HourlyWindSpeed' columns with 0
    hourly_data[['hourly_precipitation', 'hourly_wind_speed']] = hourly_data[['hourly_precipitation', 'hourly_wind_speed']].fillna(0)
    
    return hourly_data

In [ ]:
def clean_month_weather_data_daily(csv_file):  
    weather_data = pd.read_csv(csv_file)
    weather_data['DATE'] = pd.to_datetime(weather_data['DATE']).dt.date
    
     # Filter out the rows where 'REPORT_TYPE' column contains the string 'SOD'
    hourly_data = weather_data.loc[~weather_data['REPORT_TYPE'].str.contains('SOD')]

    # Select only the required columns
    hourly_data = hourly_data[['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']]
    
    # Convert the 'HourlyPrecipitation' and 'HourlyWindSpeed' columns to numeric format
    hourly_data['HourlyPrecipitation'] = pd.to_numeric(hourly_data['HourlyPrecipitation'], errors='coerce').fillna(0)
    hourly_data['HourlyWindSpeed'] = pd.to_numeric(hourly_data['HourlyWindSpeed'], errors='coerce').fillna(0)
    
    # Fill NaN values in 'HourlyPrecipitation' and 'HourlyWindSpeed' columns with 0
    hourly_data['HourlyPrecipitation']=hourly_data['HourlyPrecipitation'].fillna(0)
    hourly_data['HourlyWindSpeed']=hourly_data['HourlyWindSpeed'].fillna(0)
    
    # Aggregate the hourly data by date
    hourly_data = hourly_data.groupby('DATE', as_index=False).agg({'HourlyWindSpeed': 'mean', 
                                                                   'HourlyPrecipitation': 'sum'})
    
    # Filter the daily data and select the 'DATE', 'Sunrise', and 'Sunset' columns
    daily_data = weather_data[weather_data['REPORT_TYPE'] == 'SOD  ']
    daily_data=daily_data[['DATE', 'Sunrise', 'Sunset']]
    
    # Convert the 'Sunrise' and 'Sunset' columns to numeric format
    daily_data['Sunrise'] = pd.to_numeric(daily_data['Sunrise'], errors='coerce')
    daily_data['Sunset'] = pd.to_numeric(daily_data['Sunset'], errors='coerce')
    
    # Merge the hourly and daily data on the 'DATE' column
    daily_data = hourly_data.merge(daily_data, on='DATE', how='left')
    
    # Fill the data
    daily_data.fillna(method='ffill',inplace=True)
    daily_data.fillna(method='bfill',inplace=True)
    
    # Rename the columns.
    daily_data = daily_data.rename(columns={'DATE': 'date', 
                                            'HourlyPrecipitation': 'daily_precipitation', 
                                            'HourlyWindSpeed': 'daily_wind_speed', 
                                            'Sunrise': 'sunrise', 
                                            'Sunset': 'sunset'})
    
    return daily_data

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
daily_weather_data.head()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    datetime DATETIME,
    hourly_precipitation FLOAT,
    hourly_wind_speed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    date DATE,
    daily_wind_speed FLOAT,
    daily_precipitation FLOAT
    sunrise FLOAT
    sunset FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi
(
    id INTEGER PRIMARY KEY,
    Trip_Pickup_DateTime DATETIME,
    Trip_Dropoff_DateTime DATETIME,
    Total_Amt FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    duration FLOAT
)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    Trip_Pickup_DateTime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
);
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
with engine.connect() as connection:
    connection.execute(TAXI_TRIPS_SCHEMA)

In [ ]:
with engine.connect() as connection:
    connection.execute(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    for name, table in table_to_df_dict.items():
        table.to_sql(name, engine, if_exists='replace', index=False)

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

### Query 1

In [ ]:
QUERY_1 = """
SELECT strftime('%H', Trip_Pickup_DateTime), count(*)
FROM taxi_trips
GROUP BY strftime('%H', Trip_Pickup_DateTime)
ORDER BY count(*) DESC;
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, "Popularity of taxi rides for each hour")

In [ ]:
QUERY4 = """
    WITH hired_trips AS
    (
    SELECT 
        Trip_Pickup_DateTime, 
        distance
    FROM
        taxi_trips
    WHERE 
        strftime('%Y', Trip_Pickup_DateTime) = "2009"
UNION
    SELECT
        Trip_Pickup_DateTime, 
        distance
    FROM
        uber_trips
    WHERE 
        strftime('%Y', Trip_Pickup_DateTime) = "2009"
    )
    SELECT 
        date(Trip_Pickup_DateTime), count(*), AVG(distance)
    FROM hired_trips
    GROUP BY date(Trip_Pickup_DateTime)
    ORDER BY count(*) DESC
    LIMIT 10

"""

In [ ]:
engine.execute(QUERY_4).fetchall()

In [ ]:
write_query_to_file(QUERY_4, "Top 10 days with highest number of hired rides in 2009")

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)